In [31]:
import pandas as pd 
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
import jellyfish



In [32]:
def generate_blocking_keys(df):
    df.fillna('', inplace=True)

    ### hierarchial blocking: fileid
    # df['filename_blocking_key'] = df['id'].str.lower().str.strip()
    
    # Generate partial blocking keys from names
    df['last_name_blocking_key'] = df['entity_1_last_name'].str.lower().str[:3] + '_' + df['entity_2_last_name'].str.lower().str[:3]
    df['first_name_blocking_key'] = df['entity_1_first_name'].str.lower().str[:3] + '_' + df['entity_2_first_name'].str.lower().str[:3]
    
    # Generate a blocking key based on context similarity
    df['context_similar'] = np.where(df['context_similarity'].astype(float) > 0.9, 'context_similar', 'context_not_similar')
    
    
    # Combine all blocking keys into a single combined_blocking_key
    df['combined_blocking_key'] = df.apply(lambda x: '_'.join([
        x['first_name_blocking_key'],
        x['last_name_blocking_key'],
        x['context_similar'],
    ]), axis=1)

    df
    
    return df



df = pd.read_csv("../../ts-feature-engineering/data/output/features.csv")

df['normalized_label'] = df['label'].str.lower().str.strip()

label_mapping = {
        'easy match': 1, 
        'hard match': 1, 
        'easy non-match': 0, 
        'hard non-match': 0
    }

df['label_numeric'] = df['normalized_label'].map(label_mapping)

df = generate_blocking_keys(df)
df
df.to_csv("../data/output/ts-blocks.csv", index=False)